In [1]:
import pyomo.environ as pe  # Pyomo environment
from idaes.core import FlowsheetBlock, StateBlock
from idaes.models.unit_models import HeatExchanger
from idaes.models.unit_models.heat_exchanger import HX0DInitializer
from idaes.models.unit_models.heat_exchanger import delta_temperature_amtd_callback
from idaes.models.properties import iapws95

# Create an empty flowsheet and steam property parameter block.
model = pe.ConcreteModel()
model.fs = FlowsheetBlock(dynamic=False)
model.fs.properties = iapws95.Iapws95ParameterBlock()

# Provided data in Celsius, convert to Kelvin
T_hot_inlet_C = 662.30
T_hot_outlet_C = 192.60
T_cold_inlet_C = 20.00
T_cold_outlet_C = 496.94

T_hot_inlet = T_hot_inlet_C + 273.15  # Convert to Kelvin
T_hot_outlet = T_hot_outlet_C + 273.15  # Convert to Kelvin
T_cold_inlet = T_cold_inlet_C + 273.15  # Convert to Kelvin
T_cold_outlet = T_cold_outlet_C + 273.15  # Convert to Kelvin

# Assume constant Cp for gases (ideal gas assumption)
Cp_hot = 1.04  # Specific heat of nitrogen (N2) in kJ/kg·K
Cp_cold = 1.005  # Specific heat of air in kJ/kg·K
Cp_steam = 4.18  # Specific heat of steam in kJ/kg·K (assumed for the example)

# Reference temperature (for simplicity, assume 0°C = 273.15 K)
T_ref = 273.15

# Calculate enthalpy using Cp (simplified)
def calc_enthalpy(Cp, T):
    return Cp * (T - T_ref)

# Calculate enthalpy for the hot and cold streams
h_hot_inlet = calc_enthalpy(Cp_hot, T_hot_inlet)  # Inlet enthalpy for the hot stream (N2/steam)
h_cold_inlet = calc_enthalpy(Cp_cold, T_cold_inlet)  # Inlet enthalpy for the cold stream (air)

# Create the Heat Exchanger model
model.fs.heat_exchanger = HeatExchanger(
    delta_temperature_callback=delta_temperature_amtd_callback,
    hot_side_name="shell",
    cold_side_name="tube",
    shell={"property_package": model.fs.properties},
    tube={"property_package": model.fs.properties}
)

# Fixing inlet flow and pressure values (assuming standard conditions)
model.fs.heat_exchanger.shell_inlet.flow_mol.fix(100)  # Hot stream flow rate (mol/s)
model.fs.heat_exchanger.shell_inlet.pressure.fix(101325)  # Standard atmospheric pressure
model.fs.heat_exchanger.tube_inlet.flow_mol.fix(100)  # Cold stream flow rate (mol/s)
model.fs.heat_exchanger.tube_inlet.pressure.fix(101325)  # Standard atmospheric pressure

# Fixing enthalpies for the heat exchanger (we are using simplified calculation)
model.fs.heat_exchanger.shell_inlet.enth_mol[0].set_value(h_hot_inlet)
model.fs.heat_exchanger.tube_inlet.enth_mol[0].set_value(h_cold_inlet)

# Fixing heat exchanger parameters
model.fs.heat_exchanger.area.fix(1000)  # Heat exchanger area (m², fixed for now)
model.fs.heat_exchanger.overall_heat_transfer_coefficient[0].fix(100)  # Overall heat transfer coefficient (W/m²·K)

# Initialize the model
initializer = HX0DInitializer()
initializer.initialize(model.fs.heat_exchanger)

# Now we can calculate the outlet temperatures and enthalpies after solving the model.
# Calculate the outlet enthalpy for hot and cold streams
T_hot_outlet_optimized = pe.value(model.fs.heat_exchanger.shell.properties_out[0].temperature)
h_hot_outlet = calc_enthalpy(Cp_hot, T_hot_outlet_optimized)

T_cold_outlet_optimized = pe.value(model.fs.heat_exchanger.tube.properties_out[0].temperature)
h_cold_outlet = calc_enthalpy(Cp_cold, T_cold_outlet_optimized)

# Output the enthalpies and temperatures
print(f"Hot Stream Inlet Enthalpy: {h_hot_inlet} kJ/mol")
print(f"Cold Stream Inlet Enthalpy: {h_cold_inlet} kJ/mol")
print(f"Hot Stream Outlet Enthalpy: {h_hot_outlet} kJ/mol")
print(f"Cold Stream Outlet Enthalpy: {h_cold_outlet} kJ/mol")
print(f"Hot Stream Inlet Temperature: {T_hot_inlet_C} °C")
print(f"Hot Stream Outlet Temperature: {T_hot_outlet_C} °C")
print(f"Cold Stream Inlet Temperature: {T_cold_inlet_C} °C")
print(f"Cold Stream Outlet Temperature: {T_cold_outlet_C} °C")

2025-03-28 14:59:16 [INFO] idaes.init.fs.heat_exchanger: Initialization Completed, optimal - <undefined>
Hot Stream Inlet Enthalpy: 688.792 kJ/mol
Cold Stream Inlet Enthalpy: 20.099999999999998 kJ/mol
Hot Stream Outlet Enthalpy: 0.9048712431338026 kJ/mol
Cold Stream Outlet Enthalpy: 8.49325608546755 kJ/mol
Hot Stream Inlet Temperature: 662.3 °C
Hot Stream Outlet Temperature: 192.6 °C
Cold Stream Inlet Temperature: 20.0 °C
Cold Stream Outlet Temperature: 496.94 °C


In [2]:
# Delete the existing 'area' component before redefining it
model.fs.heat_exchanger.del_component(model.fs.heat_exchanger.area)

# Define area as a decision variable to be optimized
model.fs.heat_exchanger.area = pe.Var(within=pe.NonNegativeReals)

In [3]:
# Set the objective to minimize the heat exchanger area
model.obj = pe.Objective(expr=model.fs.heat_exchanger.area)

In [4]:
# Initialize the model
from idaes.models.unit_models.heat_exchanger import HX0DInitializer
initializer = HX0DInitializer()
initializer.initialize(model.fs.heat_exchanger)

2025-03-28 14:59:16 [INFO] idaes.init.fs.heat_exchanger: Initialization Completed, optimal - <undefined>


<InitializationStatus.Ok: 1>

In [5]:
# Solve the model using a solver (e.g., IPOPT)
solver = pe.SolverFactory('ipopt')
solver.solve(model, tee=True)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 10, 'Number of variables': 13, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.13.2\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.45131635665893555}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [6]:
# Output the results
print(f"Optimized Heat Exchanger Area: {pe.value(model.fs.heat_exchanger.area)} m^2")
# Calculate the heat duty based on the hot side
heat_duty = pe.value(model.fs.heat_exchanger.shell_inlet.flow_mol[0] * (h_hot_inlet - h_hot_outlet))
print(f"Optimized Heat Duty: {heat_duty} J/mol")

Optimized Heat Exchanger Area: 0.0 m^2
Optimized Heat Duty: 68788.71287568662 J/mol


In [7]:
import idaes_ui
import idaes_ui.fv
dir(idaes_ui.fv)
from idaes_ui.fv import visualize 

# Assign a value to the heat exchanger area before visualization
model.fs.heat_exchanger.area.set_value(pe.value(model.fs.heat_exchanger.area))

# Visualize the model
visualize(model.fs, "HeatExchangerAreaMinimization")

2025-03-28 14:59:18 [INFO] idaes.idaes_ui.fv.fsvis: Started visualization server
2025-03-28 14:59:18 [INFO] idaes.idaes_ui.fv.fsvis: Loading saved flowsheet from 'HeatExchangerAreaMinimization.json'
2025-03-28 14:59:18 [INFO] idaes.idaes_ui.fv.fsvis: Saving flowsheet to default file 'HeatExchangerAreaMinimization.json' in current directory (c:\Users\Sara\Desktop\IDAES)
2025-03-28 14:59:19 [INFO] idaes.idaes_ui.fv.fsvis: Flowsheet visualization at: http://localhost:62889/app?id=HeatExchangerAreaMinimization


VisualizeResult(store=<idaes_ui.fv.persist.FileDataStore object at 0x000001B6E0FA47C0>, port=62889, server=<idaes_ui.fv.model_server.FlowsheetServer object at 0x000001B6D8ED9F90>)